In [3]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
     

import torchvision
import torchvision.transforms as transforms

from models import *


global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')
    
batch_size = 128
model_name = "project_resnet"
model = resnet20_quant_alpha()

print(model)

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    adjust_list = [150, 225]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1        

#model = nn.DataParallel(model).cuda()
#all_params = checkpoint['state_dict']
#model.load_state_dict(all_params, strict=False)
#criterion = nn.CrossEntropyLoss().cuda()
#validate(testloader, model, criterion)

=> Building model...
ResNet_Cifar_Single8(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): QuantConv2d(
        16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()
      )
      (conv2): QuantConv2d(
        16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()
      )
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): QuantConv2d(
        16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quant

In [24]:
lr = .0005
weight_decay = .0001
epochs = 50
best_prec = 0

#model = nn.DataParallel(model).cuda()
model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
#cudnn.benchmark = True

from warmup_scheduler import GradualWarmupScheduler

base_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=epochs
)

scheduler = GradualWarmupScheduler(
    optimizer,
    multiplier=1,  
    total_epoch=10,  
    after_scheduler=base_scheduler
)

if not os.path.exists('result'):
    os.makedirs('result')
fdir = 'result/'+str(model_name)
if not os.path.exists(fdir):
    os.makedirs(fdir)
        
for epoch in range(0, epochs):
    
    train(trainloader, model, criterion, optimizer, epoch)
    
    print("Validation starts")
    prec = validate(testloader, model, criterion)

    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)

    # NEW
    scheduler.step()
    print(f"Epoch {epoch} — LR: {scheduler.get_last_lr()}")


Epoch: [0][0/391]	Time 0.316 (0.316)	Data 0.284 (0.284)	Loss 0.4332 (0.4332)	Prec 89.844% (89.844%)
Epoch: [0][100/391]	Time 0.023 (0.026)	Data 0.001 (0.004)	Loss 0.4513 (0.3701)	Prec 85.156% (87.191%)
Epoch: [0][200/391]	Time 0.023 (0.024)	Data 0.001 (0.003)	Loss 0.4260 (0.3684)	Prec 83.594% (87.084%)
Epoch: [0][300/391]	Time 0.023 (0.024)	Data 0.001 (0.002)	Loss 0.3217 (0.3647)	Prec 89.062% (87.220%)
Validation starts
Test: [0/79]	Time 0.293 (0.293)	Loss 0.4866 (0.4866)	Prec 85.156% (85.156%)
 * Prec 83.490% 
best acc: 83.490000
Epoch 0 — LR: [5e-05]
Epoch: [1][0/391]	Time 0.367 (0.367)	Data 0.337 (0.337)	Loss 0.4220 (0.4220)	Prec 84.375% (84.375%)
Epoch: [1][100/391]	Time 0.024 (0.027)	Data 0.001 (0.005)	Loss 0.3256 (0.3372)	Prec 89.844% (87.925%)
Epoch: [1][200/391]	Time 0.023 (0.025)	Data 0.001 (0.003)	Loss 0.1669 (0.3293)	Prec 96.094% (88.285%)
Epoch: [1][300/391]	Time 0.022 (0.025)	Data 0.001 (0.003)	Loss 0.3127 (0.3196)	Prec 87.500% (88.650%)
Validation starts
Test: [0/79]	Time

In [25]:
PATH = "result/project_resnet/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 8634/10000 (86%)



In [3]:
#send an input and grap the value by using prehook like HW3

In [26]:
# choose the first 8x8 QuantConv2d in layer3
layer = model.layer3[0].conv2
# print(layer)

w_bit = 4
weight_q     = layer.weight_q.detach()                           # quantized weights
weight_alpha = layer.weight_quant.state_dict()['wgt_alpha']      # alpha from layer
weight_delta = weight_alpha / ((2**(w_bit - 1)) - 1)
weight_int   = torch.round(weight_q / weight_delta).to(torch.int8)

# optional: sanity check
print(weight_int.shape)


torch.Size([8, 8, 3, 3])


In [27]:
# === grab activation of the 1st 8x8 QuantConv2d in layer3 and quantize it ===
x_bit = 4
activation = {}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

def hook_fn(module, input, output):
    # input is a tuple: (x,)
    activation["x"] = input[0].detach()   # keep on CPU for now

# register hook on the chosen conv layer
hook_handle = layer.register_forward_hook(hook_fn)

# run one batch through the model to populate activation["x"]
data_iter = iter(testloader)
images, targets = next(data_iter)        # CIFAR-10 batch
images = images.to(device)

with torch.no_grad():
    _ = model(images)

# remove hook so it doesn't affect later runs
hook_handle.remove()

# ----- quantize the captured activation -----
x = activation["x"].to(device)           # input of the 2nd conv in layer3[0]
print("x.shape:", x.shape)

x_alpha = layer.act_alpha.item()
x_delta = x_alpha / (2 ** x_bit - 1)

act_quant_fn = act_quantization(x_bit)   # from quant_layer.py
x_q = act_quant_fn(x, x_alpha)           # quantized activation (float)
x_int = torch.round(x_q / x_delta).to(torch.int32)

print("x_int.shape:", x_int.shape)
print("x_int min/max:", x_int.min().item(), x_int.max().item())


x.shape: torch.Size([128, 8, 8, 8])
x_int.shape: torch.Size([128, 8, 8, 8])
x_int min/max: 0 15


In [28]:
in_ch, out_ch = layer.in_channels, layer.out_channels
k = layer.kernel_size[0]
pad = layer.padding[0]
stride = layer.stride[0]

conv_int = torch.nn.Conv2d(in_ch, out_ch, kernel_size=k, stride=stride, padding=pad, bias=False).cuda()
print(conv_int)
conv_int.weight = torch.nn.Parameter(weight_int.to(torch.float32))

output_int = conv_int(x_int.to(torch.float32))           # output_int can be calculated with conv_int and x_int
output_recovered = output_int * (x_delta * weight_delta) # recover with x_delta and w_delta

print("output_recovered:", output_recovered.shape)

Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
output_recovered: torch.Size([128, 8, 8, 8])


In [29]:
#### input floating number / weight quantized version
conv_ref = torch.nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1, bias=False).cuda()
conv_ref.weight = torch.nn.Parameter(weight_q)
output_ref = conv_ref(x)
# print(output_ref)

In [30]:
difference = abs(output_ref - output_recovered)
print(difference.mean().item())

0.6927454471588135
